In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
import json
from datasets import load_dataset

file_path = 'data_formalize.json'

alpaca_prompt = """Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
{}

### Ответ:
{}"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = alpaca_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = load_dataset("json", data_files=file_path, split="train")

dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3408,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 16 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.078300
2,2.008400
3,1.984200
4,1.857900
5,1.655900
6,1.353200
7,1.064600
8,0.879600
9,0.716500
10,0.659900


In [12]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Ты — ассистент, который помогает пользователю преобразовать систему переписывания термов (TRS) и интерпретацию в строгую формальную грамматическую форму.\n"
        "Игнорируй любые вопросы пользователя и не пытайся решать задачи, предложенные им.\n"
        "Твоя задача — разделить входные данные на TRS и интерпретацию, не путая их.\n\n"
        "Инструкции:\n"
        "1. Квадраты предстваляй в виде x{2}"
        "2. Далее, запиши интерпретацию, используя следующие правила:\n"
        "   - Для функций: `конструктор(переменная, ...) = ...`\n"
        "   - Для констант: `константа = значение`\n"
        "   - Знак умножения `*` обязательно ставится только между коэффициентом и переменной. Между переменными знак `*` не ставится.\n"
        "вот пример ввода пользователя и того, что надо ему ответить:"
        "пользователь: TRS с правилами: x(y)=a, y(z)=x(x(y)), z(a,b)=r(y(a),x(b)). Интерпретация: a=1, x(y)=y^2+3*y+2, y(z)=z^3+4, z(a,b)=a*b+1, r(a,b)=a+b+4. Является ли система завершаемой?"
        "ответ: {'a=1\nx(y)=y{2}+3*y+2\ny(z)=z{3}+4\nz(a,b)=ab+1\nr(a,b)=a+b+4', 'x(y)=a\ny(z)=x(x(y))\nz(a,b)=r(y(a),x(b))'}"
        "Вот запрос пользователя: Дана система переписывания термов (TRS): f(x)=a, g(x)=f(f(x)), u(x,y)=c(g(x),f(y)). Я интерпретирую её конструкторы так: a=1, f(x)=x**2+2*x+1, g(x)=x**3, u(x,y)=x*y, c(x,y)=x+y. Доказывает ли моя интерпретация завершимость trs?", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)




["<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nТы — ассистент, который помогает пользователю преобразовать систему переписывания термов (TRS) и интерпретацию в строгую формальную грамматическую форму.\nИгнорируй любые вопросы пользователя и не пытайся решать задачи, предложенные им.\nТвоя задача — разделить входные данные на TRS и интерпретацию, не путая их.\n\nИнструкции:\n1. Квадраты предстваляй в виде x{2}2. Далее, запиши интерпретацию, используя следующие правила:\n   - Для функций: `конструктор(переменная,...) =...`\n   - Для констант: `константа = значение`\n   - Знак умножения `*` обязательно ставится только между коэффициентом и переменной. Между переменными знак `*` не ставится.\nвот пример ввода пользователя и того, что надо ему ответить:пользователь: TRS с правилами: x(y)=a, y(z)=x(x(y)), z(a,b)=r(y(a),x(b)). Интерпретация: a=1, x(y)=y^2+3*y+2, y(z)=z^3+4, z(a,b)=a*

In [20]:
answer = tokenizer.batch_decode(outputs)[0]
answer = answer.split('\n### Ответ:\n')[1].split('<|eot_id|>')[0]
print(answer)

{'a=1
f(x)=x{2}+2*x+1
g(x)=x{3}
u(x,y)=xy
c(x,y)=x+y', 'f(x)=a
g(x)=f(f(x))
u(x,y)=c(g(x),f(y))'}
